In [1]:
# 1
# Chance Wiese

In [56]:
# 2
import pandas as pd
import psycopg2

conn = psycopg2.connect(
    database="loftsvc", 
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432'
)

cursor = conn.cursor()

query = '''SELECT * FROM exceptionlog where fixstamp is null;'''
df = pd.read_sql(query, conn)
print(df)

    exceptid                                             record fixstamp
0          1  Elizabeth Marsh||Gettysburg|2024-01-17|721|Fau...     None
1          2  Amanda||Hudson|2024-01-17||Mighty cold sucking...     None
2          3       Tracey Thompson|doors|Hudson|2024-01-18|893|     None
3          6  Teri Perry|windows|Detroit|2024-01-20|486|Stuc...     None
4          8  Jerry Massey|plumbing|Detroit||431|No hot wate...     None
5          9  John Clark||Antietam|2024-01-21|105|Unexpected...     None
6         10  Shannon Ramos|connectivity||2024-01-22|753|Slo...     None
7         11  Court Boone|connectivity|Charleston|2024-01-22...     None
8         12  Darryl Vazquez|doors|Antietam|2024-01-23|176|D...     None
9         13  Billy Walker||Antietam|2024-01-24|146|Experien...     None
10        14  Laurie Lopez|electrical||2024-01-25|413|Hopele...     None
11        15  Shane-O|electrical|Frederick|2024-01-26|685|Da...     None
12        16        Sandra Garcia|||2024-01-26||Hav

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/2144117380.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3]:
# 3
print("Rhonda Carter: Building name is missing. Could use their unit number to find what building they live in.")
print("Willie Greene: Type is missing. We can look for keywords like internet in the description to find a new one.") 
print("Jerry Massey: Record is missing the date field. Can use the previous record's date to get a close estimate.")
print("Gabriela Carter: Outlets is not a valid type. Can set it to electrical.")

Rhonda Carter: Building name is missing. Could use their unit number to find what building they live in.
Willie Greene: Type is missing. We can look for keywords like internet in the description to find a new one.
Jerry Massey: Record is missing the date field. Can use the previous record's date to get a close estimate.
Gabriela Carter: Outlets is not a valid type. Can set it to electrical.


In [4]:
# 4
cursor.execute('''
ALTER TABLE servreq 
ADD COLUMN IF NOT EXISTS exceptid INTEGER 
REFERENCES exceptionlog(exceptid);
''')

conn.commit()

query = '''
SELECT srid, exceptid
FROM servreq
LIMIT 3;
'''

df_check = pd.read_sql(query, conn)
print(df_check)

   srid exceptid
0     1     None
1     2     None
2     3     None


/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/103347601.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_check = pd.read_sql(query, conn)


In [45]:
# 5
from datetime import datetime
last_valid_date = None

for x in df.index:
    exceptid = df['exceptid'].loc[x]
    record = df['record'].loc[x].split('|')
    customer = record[0]
    type = record[1]
    building = record[2]
    date = record[3]
    unit = record[4]
    description = record[5]
    
    issues = set()
    can_fix = True
    
    # print(f"\nExceptID: {id}")
    # print(f"Customer: {customer}")
    # print(f"Type: {type}")
    # print(f"Building: {building}")
    # print(f"Date: {date}")
    # print(f"Unit: {unit}")
    # print(f"Description: {description}")
    
    # Customer
    tenants = pd.read_sql("SELECT tenantid, CONCAT(fname, ' ', lname) AS full_name FROM tenant;", conn)
    tenant_names = tenants['full_name'].tolist()
    
    if ' ' not in customer and customer:
        issues.add('One Name')
        can_fix = False
    elif customer not in tenant_names:
        issues.add('Unknown Name')
        can_fix = False
    else:
        tenantid = tenants.loc[tenants['full_name'] == customer, 'tenantid'].iloc[0]
        
    # Type
    types = pd.read_sql("SELECT * FROM reqtype", conn)
    valid_types = ['connectivity', 'doors', 'electrical', 'hvac', 'plumbing', 'windows', 'other']
    
    if type not in valid_types or not type:
        if 'internet' in description.lower():
            rtid = types.loc[types['typename'] == 'connectivity', 'rtid'].iloc[0] # 1
        elif type == 'outlets':
            rtid = types.loc[types['typename'] == 'electrical', 'rtid'].iloc[0] # 3
        else:
            issues.add('Unknown Type')
            can_fix = False
    else:
        rtid = types.loc[types['typename'] == type, 'rtid'].iloc[0]
    
    # Building
    valid_buildings = ['Antietam', 'Bull Run', 'Charleston', 'Detroit', 'Elkins Ferry', 'Fredericksburg', 'Gettysburg', 'Hudson']

    if building not in valid_buildings or not building:
        if not unit:
            issues.add('No Unit #')
            can_fix = False
        try:
            unit = int(unit)
            bldgid = pd.read_sql(f'''
            SELECT bldgid
            FROM building
            WHERE CAST(split_part(unitrange, '-', 1) AS INTEGER) <= {unit}
            AND CAST(split_part(unitrange, '-', 2) AS INTEGER) >= {unit};
            ''', conn)['bldgid'].iloc[0]
        except (ValueError, IndexError):
            if 'No Unit #' not in issues:
                issues.add('Bad Unit #')
            can_fix = False
    else:
        bldgid = pd.read_sql(f"SELECT bldgid FROM building WHERE bldgname = '{building}'", conn)['bldgid'].iloc[0]
    
    # Date
    if not date:
        if exceptid == 1:
            issues.add('No Date')
            can_fix = False
        else: 
            date = last_valid_date
    else:
        try:
            date = datetime.strptime(date, '%Y-%m-%d')
            if date.year != datetime.now().year or date > datetime.now():
                if exceptid == 1:
                    issues.add('No Date')
                    can_fix = False
                date = last_valid_date
            else:
                date = date.strftime('%Y-%m-%d')
                last_valid_date = date
                
        except ValueError:
            if exceptid == 1:
                issues.add('No Date')
                can_fix = False
            date = last_valid_date
            
    # Unit
    if not unit:
        issues.add('No Unit #')
        can_fix = False
    
    # Description
    if not description:
        issues.add('No Description')
        can_fix = False
        
        
    if not can_fix:
        print(f"Can't Fix ({', '.join(issues)}): {record}")
    else:
        print(f'''Can fix record with values:
        tenantid = {tenantid},
        rtid = {rtid},
        bldgid = {bldgid},
        reqdate = '{date}',
        unit = {unit},
        reqdescription = '{description}',
        exceptid = {exceptid}''')

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/3762647138.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tenants = pd.read_sql("SELECT tenantid, CONCAT(fname, ' ', lname) AS full_name FROM tenant;", conn)
/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/3762647138.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  types = pd.read_sql("SELECT * FROM reqtype", conn)
/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/3762647138.py:74: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAl

Can't Fix (Unknown Type): ['Elizabeth Marsh', '', 'Gettysburg', '2024-01-17', '721', 'Faucet dripping, dripping forever, dripping highly, morosely.']
Can't Fix (No Unit #, One Name, Unknown Type): ['Amanda', '', 'Hudson', '2024-01-17', '', 'Mighty cold sucking from thermostat; @#$%*&! fan.']
Can't Fix (No Description): ['Tracey Thompson', 'doors', 'Hudson', '2024-01-18', '893', '']


/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/3762647138.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bldgid = pd.read_sql(f'''


Can fix record with values:
        tenantid = 455,
        rtid = 5,
        bldgid = 2,
        reqdate = '2024-01-18',
        unit = 268,
        reqdescription = 'Aircon sounds like a ghost lives inside it.',
        exceptid = 4
Can fix record with values:
        tenantid = 593,
        rtid = 1,
        bldgid = 3,
        reqdate = '2024-01-19',
        unit = 331,
        reqdescription = 'Internet running at 270 bits per second.',
        exceptid = 5
Can't Fix (Unknown Name): ['Teri Perry', 'windows', 'Detroit', '2024-01-20', '486', 'Stuck and injured @#$%*&!, lock @#$%*&! handle will not open.']
Can fix record with values:
        tenantid = 504,
        rtid = 2,
        bldgid = 3,
        reqdate = '2024-01-21',
        unit = 345,
        reqdescription = 'Door bashed with an axe bashed with an axe trapped locked inside.',
        exceptid = 7
Can fix record with values:
        tenantid = 255,
        rtid = 6,
        bldgid = 4,
        reqdate = '2024-01-21',
     

In [58]:
# 6
query = '''SELECT * FROM exceptionlog where fixstamp is null;'''
df = pd.read_sql(query, conn)

last_valid_date = None  # For tracking the last valid date

for x in df.index:
    exceptid = df['exceptid'].loc[x]
    record = df['record'].loc[x].split('|')
    customer = record[0]
    type = record[1]
    building = record[2]
    date = record[3]
    unit = record[4]
    description = record[5]
    
    can_fix = True
    
    # Customer
    tenants = pd.read_sql("SELECT tenantid, CONCAT(fname, ' ', lname) AS full_name FROM tenant;", conn)
    tenant_names = tenants['full_name'].tolist()
    
    if ' ' not in customer and customer:
        can_fix = False
    elif customer not in tenant_names:
        can_fix = False
    else:
        tenantid = tenants.loc[tenants['full_name'] == customer, 'tenantid'].iloc[0]
        
    # Type
    types = pd.read_sql("SELECT * FROM reqtype", conn)
    valid_types = ['connectivity', 'doors', 'electrical', 'hvac', 'plumbing', 'windows', 'other']
    
    if type not in valid_types or not type:
        if 'internet' in description.lower():
            rtid = types.loc[types['typename'] == 'connectivity', 'rtid'].iloc[0]
        elif type == 'outlets':
            rtid = types.loc[types['typename'] == 'electrical', 'rtid'].iloc[0]
        else:
            can_fix = False
    else:
        rtid = types.loc[types['typename'] == type, 'rtid'].iloc[0]
        
    # Building
    if building not in ['Antietam', 'Bull Run', 'Charleston', 'Detroit', 'Elkins Ferry', 'Fredericksburg', 'Gettysburg', 'Hudson'] or not building:
        if not unit:
            can_fix = False
        try:
            unit = int(unit)
            bldgid = pd.read_sql(f'''
            SELECT bldgid FROM building 
            WHERE CAST(split_part(unitrange, '-', 1) AS INTEGER) <= {unit}
            AND CAST(split_part(unitrange, '-', 2) AS INTEGER) >= {unit};
            ''', conn)['bldgid'].iloc[0]
        except (ValueError, IndexError):
            can_fix = False
    else:
        bldgid = pd.read_sql(f"SELECT bldgid FROM building WHERE bldgname = '{building}'", conn)['bldgid'].iloc[0]
        
    # Date
    if not date:
        if exceptid == 1:
            can_fix = False
        else:
            date = last_valid_date
    else:
        try:
            date_obj = datetime.strptime(date, '%Y-%m-%d')
            if date_obj.year != datetime.now().year or date_obj > datetime.now():
                if exceptid == 1:
                    can_fix = False
                date = last_valid_date
            else:
                date = date_obj.strftime('%Y-%m-%d')
                last_valid_date = date
        except ValueError:
            if exceptid == 1:
                can_fix = False
            date = last_valid_date
            
    # If record can be fixed
    if can_fix and description and unit:
        query = f'''
        INSERT INTO servreq (tenantid, rtid, bldgid, reqdate, unit, reqdescription, exceptid)
        VALUES ({tenantid}, {rtid}, {bldgid}, '{date}', {unit}, '{description}', {exceptid})
        '''
        cursor.execute(query)
        
        query = f'''
        UPDATE exceptionlog 
        SET fixstamp = CURRENT_TIMESTAMP 
        WHERE exceptid = {exceptid}
        '''
        cursor.execute(query)
        
        conn.commit()
        recordvalue = f"tenantid={tenantid}, rtid={rtid}, bldgid={bldgid}, reqdate='{date}', unit={unit}, description='{description}', exceptid={exceptid}"
        print(f"Record Added: {recordvalue}")
    else:
        print(f"Can't Fix: {record}")

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/996914472.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/996914472.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tenants = pd.read_sql("SELECT tenantid, CONCAT(fname, ' ', lname) AS full_name FROM tenant;", conn)
/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/996914472.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  types = pd.read_

Can't Fix: ['Elizabeth Marsh', '', 'Gettysburg', '2024-01-17', '721', 'Faucet dripping, dripping forever, dripping highly, morosely.']
Can't Fix: ['Amanda', '', 'Hudson', '2024-01-17', '', 'Mighty cold sucking from thermostat; @#$%*&! fan.']
Can't Fix: ['Tracey Thompson', 'doors', 'Hudson', '2024-01-18', '893', '']
Can't Fix: ['Teri Perry', 'windows', 'Detroit', '2024-01-20', '486', 'Stuck and injured @#$%*&!, lock @#$%*&! handle will not open.']
Record Added: tenantid=255, rtid=6, bldgid=4, reqdate='2024-01-20', unit=431, description='No hot water. 11 days. Hope dwindling.', exceptid=8
Can't Fix: ['John Clark', '', 'Antietam', '2024-01-21', '105', 'Unexpected a/c incursions.']


/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/996914472.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bldgid = pd.read_sql(f'''


Record Added: tenantid=538, rtid=1, bldgid=7, reqdate='2024-01-22', unit=753, description='Slow download of wireless, hopelessness of websites; Internet law suit.', exceptid=10
Can't Fix: ['Court Boone', 'connectivity', 'Charleston', '2024-01-22', '', 'Slow download wireless for hopeless websites for.']
Record Added: tenantid=116, rtid=2, bldgid=1, reqdate='2024-01-23', unit=176, description='Door wont shut, door refuses the shutting, the door.', exceptid=12
Can't Fix: ['Billy Walker', '', 'Antietam', '2024-01-24', '146', 'Experiencing thermostat wheeze.']
Record Added: tenantid=495, rtid=3, bldgid=4, reqdate='2024-01-25', unit=413, description='Hopeless light switches demean us all.', exceptid=14
Can't Fix: ['Shane-O', 'electrical', 'Frederick', '2024-01-26', '685', 'Darkness, switches of darkness, the fridge LED wire explodes.']
Can't Fix: ['Sandra Garcia', '', '', '2024-01-26', '', 'Having trouble.']
Can't Fix: ['Sandra Glass', 'plumbing', 'Bull Run', '2024-01-26', '', 'With plunger

In [60]:
# 7
query = '''
SELECT srid, tenantid, rtid, bldgid, reqdate, unit, exceptid 
FROM servreq 
WHERE exceptid IS NOT NULL 
ORDER BY srid;
'''
fixed_records = pd.read_sql(query, conn)
print("\nFixed Records in ServReq:")
print(fixed_records)


Fixed Records in ServReq:
   srid  tenantid  rtid  bldgid     reqdate  unit  exceptid
0   115       255     6       4  2024-01-20   431         8
1   116       538     1       7  2024-01-22   753        10
2   117       116     2       1  2024-01-23   176        12
3   118       495     3       4  2024-01-25   413        14
4   119       610     3       2  2024-01-26   293        19
5   120       455     5       2  2024-01-18   268         4
6   121       593     1       3  2024-01-19   331         5
7   122       504     2       3  2024-01-21   345         7


/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/4061863984.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fixed_records = pd.read_sql(query, conn)


In [62]:
# 8
query = '''
SELECT exceptid, record, fixstamp 
FROM exceptionlog 
WHERE fixstamp IS NOT NULL 
ORDER BY exceptid;
'''
fixed_exceptions = pd.read_sql(query, conn)
print("\nFixed Records in ExceptionLog:")
print(fixed_exceptions)


Fixed Records in ExceptionLog:
   exceptid                                             record  \
0         4  Rhonda Carter|hvac||2024-01-18|268|Aircon soun...   
1         5  Willie Greene||Charleston|2024-01-19|331|Inter...   
2         7  Victoria Ward|doors||2024-01-21|345|Door bashe...   
3         8  Jerry Massey|plumbing|Detroit||431|No hot wate...   
4        10  Shannon Ramos|connectivity||2024-01-22|753|Slo...   
5        12  Darryl Vazquez|doors|Antietam|2024-01-23|176|D...   
6        14  Laurie Lopez|electrical||2024-01-25|413|Hopele...   
7        19  Gabriela Carter|outlets|Bull Run|2024-01-26|29...   

                    fixstamp  
0 2024-10-23 07:12:19.272702  
1 2024-10-23 07:12:19.846583  
2 2024-10-23 07:12:20.276736  
3 2024-10-23 07:12:03.558648  
4 2024-10-23 07:12:16.471747  
5 2024-10-23 07:12:17.131237  
6 2024-10-23 07:12:17.737499  
7 2024-10-23 07:12:18.280289  


/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_56612/3681207989.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fixed_exceptions = pd.read_sql(query, conn)


In [64]:
# Close Connection
cursor.close()
conn.close()